# Assignment 3

In [1]:
import numpy as np 
import matplotlib as mp
import matplotlib.pyplot as plt

from numpy import sin, pi
from scipy.optimize import curve_fit 
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Question 1: Polynomial fit

We want to find a fourth degree polynomial fit that goes through the points $(-2, 0)$, $(-1, -1)$, $(0, 0)$, $(1, 1)$, and $(2, 0)$. 
To do this, we set up a system of linear equations:

$$\vec{y} = X \vec{a}$$

$$\text{where} \left\{ \begin{array}{ll}
            \vec{y}  & \text{is the vector of the "y" values} \\
            \vec{X}  & \text{is the matrix of the "x" values} \\
            \vec{a}  & \text{is the vector of the coefficients of the polynomial} \\
            \end{array} \right.$$      

Our polynomial will have the form:

$$y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + a_4 x^4$$

Substituting the given values into the equation gives us: 
$$
\begin{bmatrix}
    0\\
    -1\\
    0\\
    1\\
    0\\
\end{bmatrix}
=
\begin{bmatrix}
    1 & -2 & 4 & -8  & 16 \\
    1 & -1 & 1 & -1 & 1 \\
    1 & 0 & 0 & 0 & 0 \\
    1 & 1 & 1 & 1 & 1 \\
    1 & 2 & 4 & 8  & 16 \\
\end{bmatrix}
\begin{bmatrix}
    a_0 \\
    a_1 \\
    a_2 \\
    a_3 \\
    a_4 \\
\end{bmatrix}$$

To find $\vec{a}$ we first find the inverse of $X$, then $\vec{a} = X^{-1} \vec{y}$ 

In [2]:
P = np.array([[-2, 0], [-1, -1], [0, 0], [1, 1], [2,0]])
x_space = np.linspace(-2, 2, 100)

In [27]:
# Generate matrix "X"
def X(p=P):
    X = np.zeros((p.shape[0], p.shape[0]))
    for i in range(0, p.shape[0]):
        X[i] += p[:,0] ** i
    return X.T

# Invert "X"
def inv_X(p=P):
    inv_x = np.linalg.inv(X(p))
    return inv_x

# Find vector "a"
def vec_a(p=P):
    a =  inv_X(p) @ p[:,1]
    return a

In [40]:
# Get our vector "a"
a = vec_a()

# Equation for "y"
def y(X, A=a):
    y = 0
    for i in range(0, np.size(A)):
        y += a[i] * X**i
    return y

In [50]:
# Plot it out: 
# ============

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.scatter(P[:,0], P[:,1], c='black', label="Data points", zorder=5)
ax.plot(x_space, sin(pi*x_space / 2), label=r"$y=\sin \left(\frac{2 x}{\pi}\right)$")
ax.plot(x_space, y(x_space), label=r"$y$ = %5.3f + %5.3f $x$ + %5.3f $x^2$ + %5.3f $x^3$ + %5.3f $x^4$" % tuple(a))
ax.set_ylabel(r"$y$", rotation=0, fontsize=14)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_title(r"Polynomial fit", fontsize=16)

plt.legend()
plt.show()

<Figure size 720x432 with 1 Axes>

## Question 2: Cubic spline fit

We want to find a cubic spline $S(x)$ with boundary conditions $S'(-2)=S'(2)$ and $S''(-2)=S''(2)$.

As we have five points, we need to find four equations for the functions between these points. This gives us the equation of the cubic spline: 

$$\left\{ \begin{array}{ll}
            S_0(x_0) = a_0 + b_0 x_0 + c_0 x_0^2 + d_0 x_0^3 \\
            S_0(x_1) = a_0 + b_0 x_1 + c_0 x_1^2 + d_0 x_1^3 \\
            S_1(x_1) = a_1 + b_1 x_1 + c_1 x_1^2 + d_1 x_1^3 \\
            S_1(x_2) = a_1 + b_1 x_2 + c_1 x_2^2 + d_1 x_2^3 \\
            S_2(x_2) = a_2 + b_2 x_2 + c_2 x_2^2 + d_2 x_2^3 \\
            S_2(x_3) = a_2 + b_2 x_3 + c_2 x_3^2 + d_2 x_3^3 \\
            S_3(x_3) = a_3 + b_3 x_3 + c_3 x_3^2 + d_3 x_3^3 \\
            S_3(x_4) = a_3 + b_3 x_4 + c_3 x_4^2 + d_3 x_4^3 \\
            \end{array} \right.$$    

We want the spline to have continuous first derivatives: 

$$\left\{ \begin{array}{ll}
            S'_0(x_1) = b_0 + c_0 x_1 + d_0 x_1^2 = b_1 + c_1 x_1 + d_1 x_1^2 = S'_1(x_1)\\
            S'_1(x_2) = b_1 + c_1 x_2 + d_1 x_2^2 = b_2 + c_2 x_2 + d_2 x_2^2 = S'_2(x_2)\\
            S'_2(x_3) = b_2 + c_2 x_3 + d_2 x_3^2 = b_3 + c_3 x_3 + d_3 x_3^2 = S'_3(x_3)\\
            \end{array} \right.$$    
            
The boundary conditions require that:
$$S'_0(x_0) = b_0 + c_0 x_0 + d_0 x_0^2 = b_3 + c_3 x_4 + d_3 x_4^2 = S'_3(x_4)$$

For a natural spline, we want the second order derivatives to be continuous and to be zero at the ends: 

$$\left\{ \begin{array}{ll}
            S''_0(x_1) = c_0 + d_0 x_1 = c_1 + d_1 x_1 = S''_1(x_1)\\
            S''_1(x_2) = c_1 + d_1 x_2 = c_2 + d_2 x_2 = S''_2(x_2)\\
            S''_2(x_3) = c_2 + d_2 x_3 = c_3 + d_3 x_3 = S''_3(x_3)\\
            S_0''(x_0) = c_0 + d_0 x_0 = c_3 + d_3 x_4 = S_3''(x_4) = 0 \\
            \end{array} \right.$$ 

Substituting the values into our equations, we have 13 equations to solve for 12 unknowns: 

$$\left\{ \begin{array}{ll}
            S_0(-2) = 0 = a_0 - 2 b_0 + 4 c_0 - 8 d_0  \\
            S_0(-1) = -1 = a_0 - b_0 + c_0 - d_0 \\
            S_1(-1) = -1 = a_1 - b_1 + c_1 - d_1 \\
            S_1(0) = 0 =a_1 \\
            S_2(0) = 0 =a_2 \\
            S_2(1) = 1 =  a_2 + b_2 + c_2 + d_2 \\
            S_3(1) = 1 =  a_3 + b_3 + c_3 + d_3 \\
            S_3(2) = 0 = a_3 + 2 b_3 + 4 c_3 + 8 d_3 \\
            S'_0(-1) = S'_1(-1) = b_0 - c_0 + d_0 = b_1 - c_1 + d_1 \\
            S'_1(0) = S'_2(0) = b_1 = b_2\\
            S'_2(1) = S'_3(1) = b_2 + c_2 + d_2 = b_3 + c_3 + d_3 \\
            S'_0(-2) = S'_3(2) = b_0 - 2 c_0 + 4 d_0 = b_3 + 2 c_3 + 4 d_3 \\
            S''_0(-1) = S''_1(-1) = c_0 - d_0 = c_1 - d_1 \\
            S''_1(0) = S''_2(0) = c_1 = c_2 \\
            S''_2(1) = S''_3(1) = c_2 + d_2 = c_3 + d_3 \\
            S_0''(-2) = S_3''(2) = c_0 - 2 d_0 = c_3 + 2 d_3 = 0 \\
            \end{array} \right.$$    

We can see straight away that $a_1 = a_2 = 0$ and $c_0 - 2 d_0 = c_3 + 2 d_3 = 0$, therefore, substitute these into the expressions: 

$$\left\{ \begin{array}{ll}
            a_0 - 2 b_0 + 4 c_0 - 8 d_0 = 0 \\
            a_0 - b_0 + c_0 - d_0 = -1\\
            - b_1 + c_1 - d_1 = -1\\
            b_2 + c_2 + d_2 = 1\\
            a_3 + b_3 + c_3 + d_3 = 1\\
            a_3 + 2 b_3 + 4 c_3 + 8 d_3 = 0\\
            b_0 - c_0 + d_0 = b_1 - c_1 + d_1 \\
            b_1 = b_2\\
            b_2 + c_2 + d_2 = b_3 + c_3 + d_3\\
            b_0 = b_3\\
            c_0 - d_0 = c_1 - d_1 \\
            c_1 = c_2\\
            c_2 + d_2 = c_3 + d_3\\
            c_0 - 2 d_0 = c_3 + 2 d_3 = 0 \\
            \end{array} \right.$$    

Now, let's continue by writing these out on paper, so please see attached hand written copy.

\newpage

In [117]:
# Plot it out:
# ============

fig, ax2 = plt.subplots(1, 1, figsize=(10, 6))

ax2.scatter(P[:,0], P[:,1], c='black', label="Data points", zorder=5)
ax2.plot(x_space, sin(pi*x_space / 2), label=r"$y=\sin \left(\frac{2 x}{\pi}\right)$")

ax2.set_ylabel(r"$y$", rotation=0, fontsize=14)
ax2.set_xlabel(r"$x$", fontsize=1)
ax2.set_title(r"Cubic spline fit")

plt.ylim(-1.75, 1.25)
plt.legend()
plt.show()

<Figure size 720x432 with 1 Axes>

## Question 3

In [113]:
paths = 100
time = 50

# Change in steps for random walk
RW = np.random.randint(0, 2, size=(paths, time+1)) * 2 - 1
RW[:, 0] = 0

# Position of the random walkers
RW_pos = np.cumsum(RW, axis=1)

# Time
t = np.arange(0, np.shape(RW_pos)[1], 1)

In [114]:
fig, ax3 = plt.subplots(1, 1, figsize=(10, 4))

for i in range(0, paths):
    plt.plot(t, RW_pos[i, :])
    i += 1

plt.title(r"Positions for {0} random walkers for a time of {1}".format(paths, time), fontsize=16)    
plt.xlabel(r"Time", fontsize=14)
plt.ylabel(r"Position", fontsize=14)
plt.show()

<Figure size 720x288 with 1 Axes>

#### Plot the variance for each time step

In [115]:
# Find the variance
var = np.nanvar(RW_pos, axis=0)

In [120]:
fig, ax4 = plt.subplots(1, 1, figsize=(10, 4))

ax4.plot(t, var**0.5)

plt.title(r"Standard deviations at each time step for {0} random walkers for a time of {1}".format(paths, time), fontsize=16)    
plt.xlabel(r"Time", fontsize=14)
plt.ylabel(r"Standard deviation", fontsize=14)
plt.show()

<Figure size 720x288 with 1 Axes>

In [136]:
# Define function
# ===============
def sdev_fit(t, a, r):
    f = a * t**r
    return f

In [141]:
# Fit the curve
# =============

popt, pcov = curve_fit(sdev_fit, t, var**0.5)

In [144]:
fig, ax5 = plt.subplots(1, 1, figsize=(10, 4))

ax5.plot(t, var**0.5)
ax5.plot(t, sdev_fit(t, *popt), '-', label=r'Fit ($\sigma = a n^r$): $a$=%5.3f, $r$=%5.3f' % tuple(popt))

plt.title(r"Standard deviations at each time step for {0} random walkers for a time of {1}".format(paths, time), fontsize=16)    
plt.xlabel(r"Time", fontsize=14)
plt.ylabel(r"Standard deviation ($\sigma$)", fontsize=14)
plt.legend()
plt.show()

<Figure size 720x288 with 1 Axes>